<a href="https://colab.research.google.com/github/godelstheory/moz-raptor-analysis/blob/master/analysis/POC2_comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
project_id = 'moz-fx-data-bq-data-science'

In [0]:
from google.cloud import bigquery

client = bigquery.Client(project=project_id)


# df = client.query('''
#   select time_scheduled, origin_revision
# from `trink-sandbox.ci_timing.poc2`
# where origin_project = 'mozilla-central'
# limit 1000000
# ''').to_dataframe()

df = client.query('''
  select origin_revision, min(time_scheduled) as td_min, 
    max(time_scheduled) as td_max
  from `trink-sandbox.ci_timing.poc2`
  where origin_project = 'mozilla-central'
  GROUP BY 1
  limit 100
''').to_dataframe()

In [0]:
df.shape

(51, 3)

In [0]:
df.head()

,origin_revision,td_min,td_max
0,5a63f841eacb1632a595c30bfb3d81636a2cde8d,2019-05-23 15:55:39.791000+00:00,2019-05-26 16:41:21.797000+00:00
1,4a74609752d2e77e4be401e92978c9c32a842b40,2019-05-19 21:37:59.895000+00:00,2019-05-20 06:47:04.416000+00:00
2,893e1d6cad95e3e7ffdb3d0daf4a26fcea4a68bd,2019-05-26 16:22:50.420000+00:00,2019-05-26 21:10:15.334000+00:00
3,d8321d0b2c5b43933c5e9f201934ec7d4c3a65c0,2019-05-28 17:28:38.606000+00:00,2019-05-28 22:36:05.763000+00:00
4,840b7106d8ae3158aeba8268d2ac0b40c3682bcb,2019-05-23 04:42:26.182000+00:00,2019-05-26 16:41:43.147000+00:00


In [0]:
# df.time_scheduled.nunique()

In [0]:
df.origin_revision.nunique()

51

In [0]:
import requests

revision_url = 'https://treeherder.mozilla.org/api/project/mozilla-central/push/?revision={}'
headers = {'User-Agent': 'CoreyDowHygelundTalos/1.0'}

def get_push_timestamp(x):
  req = requests.get(revision_url.format(x), headers=headers)
  req_json = json.loads(req.text)
  return req_json['results'][0]['push_timestamp']

In [0]:
push_timestamps = {x: get_push_timestamp(x) for x in df.origin_revision.unique()}

In [0]:
push_timestamps

{'0ee76c4d554c86d4af28f4cc92d2a19666970336': 1558905173,
 '199eaff06ecd465e8cae54496369d147edb48f3e': 1558626735,
 '1ae707852b608ea77dc82c892f25e169cbc316b5': 1558129393,
 '257f2c96cef502a1d674df56c8e39d76d8ed4d89': 1558475152,
 '267ddc3595fe2147c72e97e97cdd25f3a7519fa9': 1558496458,
 '29bdbbe89264b91d1bd4205c45536c9222a8d6ee': 1558389328,
 '29c76bc4b5901f6fd331435e6db5ff47bcde04bb': 1558993992,
 '3afbce493ca6c90a252c2825b23486974087b352': 1558110850,
 '3c0f78074b727fbae112b6eda111d4c4d30cc3ec': 1558456225,
 '5a63f841eacb1632a595c30bfb3d81636a2cde8d': 1558626911,
 '5cc220ddf028de011a922042ee9ba691b94d055d': 1559038400,
 '5d3e1ea7769357bce7297b83be3863034bcf656e': 1558734599,
 '5f95b3f2ea44723ba6a8c41a4b27c88032df709f': 1558517666,
 '7c540586aedbc69e75649ab34fbaaceee912bebd': 1558129261,
 '8388f1163e54676201327b95dcb1da7b2f920b39': 1558863297,
 '840b7106d8ae3158aeba8268d2ac0b40c3682bcb': 1558586519,
 '893e1d6cad95e3e7ffdb3d0daf4a26fcea4a68bd': 1558887743,
 '8e80b2082da53556d32efa82121b2

In [0]:
df['ts_min'] = df['td_min'].astype('int64')//1e9
df['ts_max'] = df['td_max'].astype('int64')//1e9

In [0]:
import pandas as pd

# def get_vals(x):
#   ts_min = x.timestamp_scheduled.min()
#   ts_max = x.timestamp_scheduled.max()
#   ts_push = push_timestamps[x.origin_revision.iloc[0]]
#   ts_min_delta = ts_min-ts_push
#   ts_max_delta = ts_max-ts_push
#   return pd.Series({'min': ts_min, 'max': ts_max, 'push': ts_push, 
#                     'delta_min': ts_min_delta/60, 'delta_max': ts_max_delta/60})

# df.groupby('origin_revision').apply(get_vals)
df['push'] = df.origin_revision.apply(lambda x: push_timestamps[x])
df['delta_min'] = (df['ts_min'] - df['push'])/60
df['delta_max'] = (df['ts_max'] - df['push'])/60

In [0]:
df[['origin_revision', 'push', 'delta_min', 'delta_max']].to_csv('revision_task_date_diff.csv')

In [0]:
df[['origin_revision', 'push', 'delta_min', 'delta_max']]

,origin_revision,push,delta_min,delta_max
0,5a63f841eacb1632a595c30bfb3d81636a2cde8d,1558626911,0.466667,4366.166667
1,4a74609752d2e77e4be401e92978c9c32a842b40,1558301827,0.866667,549.950000
2,893e1d6cad95e3e7ffdb3d0daf4a26fcea4a68bd,1558887743,0.450000,287.866667
3,d8321d0b2c5b43933c5e9f201934ec7d4c3a65c0,1559064501,0.283333,307.733333
4,840b7106d8ae3158aeba8268d2ac0b40c3682bcb,1558586519,0.450000,5039.733333
5,199eaff06ecd465e8cae54496369d147edb48f3e,1558626735,0.583333,4233.850000
6,0ee76c4d554c86d4af28f4cc92d2a19666970336,1558905173,2.966667,200.900000
7,97dae745c1b3ef2292127ba1c4e90b1345c8f576,1558346058,0.500000,306.283333
8,8388f1163e54676201327b95dcb1da7b2f920b39,1558863297,0.416667,251.550000
9,9617ee5d9b886a899730077b207655222d27867a,1558516221,0.316667,172.350000


In [0]:
from google.colab import files

files.download('revision_task_date_diff.csv')